  1) 알콜(Alcohol)
  2) 말산(Malic acid)
  3) 회분(Ash)
  4) 회분의 알칼리도(Alcalinity of ash)
  5) 마그네슘(Magnesium)
  6) 총 폴리페놀(Total phenols)
  7) 플라보노이드 폴리페놀(Flavanoids)
  8) 비 플라보노이드 폴리페놀(Nonflavanoid phenols)
  9) 프로안토시아닌(Proanthocyanins)
  10) 색상의 강도(Color intensity)
  11) 색상(Hue)
  12) 희석 와인의 OD280/OD315 비율 (OD280/OD315 of diluted wines)
  13) 프롤린(Proline)

In [1]:
from sklearn.datasets import load_wine
datasets = load_wine()

X = datasets.data

y = datasets.target

In [2]:
import pandas as pd

df = pd.DataFrame(datasets.data, columns=datasets.feature_names)
target = pd.Series(datasets.target, dtype="category")
df['class'] = target
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


data split

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.3,random_state=2020)

정규화 및 tensor로 매핑

In [5]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()

standardScaler.fit(X_train)

X_scaling_train = standardScaler.transform(X_train)
X_scaling_test = standardScaler.transform(X_test)

In [6]:
import torch 

X_train_tensor = torch.tensor(X_scaling_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_scaling_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

모델 정의

In [7]:
import torch
import torch.nn as nn

class LogisticRegression(nn.Module):
    def __init__(self, input_size, output_size):
        #super(LogisticRegression, self).__init__()
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
        #self.softmax = nn.Softmax()
    
    def forward(self, X):
        output = self.linear(X)
    
        return output

model = LogisticRegression(13, 3)

loss function, optimizer 정의

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

학습

In [9]:
epochs = 20000
total = 0
correct = 0
#model.train()
for epoch in range(epochs):
    total = 0
    correct = 0
    
    
    prediction = model(X_train_tensor)
   
    loss = criterion(prediction, y_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    _, predicted = torch.max(prediction.data, 1)
    total += y_train_tensor.size(0)
    correct += (predicted == y_train_tensor).sum().item()
    
    if epoch % 100 == 0:
        print(f'Epoch[{epoch}/{epochs}] Loss:{round(loss.item(), 3)} Acc: {round(100 * correct / total, 3)}')

Epoch[0/20000] Loss:1.202 Acc: 31.452
Epoch[100/20000] Loss:1.016 Acc: 45.968
Epoch[200/20000] Loss:0.873 Acc: 65.323
Epoch[300/20000] Loss:0.763 Acc: 76.613
Epoch[400/20000] Loss:0.679 Acc: 84.677
Epoch[500/20000] Loss:0.612 Acc: 85.484
Epoch[600/20000] Loss:0.559 Acc: 87.903
Epoch[700/20000] Loss:0.516 Acc: 89.516
Epoch[800/20000] Loss:0.48 Acc: 90.323
Epoch[900/20000] Loss:0.449 Acc: 91.129
Epoch[1000/20000] Loss:0.423 Acc: 91.935
Epoch[1100/20000] Loss:0.401 Acc: 93.548
Epoch[1200/20000] Loss:0.381 Acc: 94.355
Epoch[1300/20000] Loss:0.364 Acc: 94.355
Epoch[1400/20000] Loss:0.348 Acc: 94.355
Epoch[1500/20000] Loss:0.334 Acc: 94.355
Epoch[1600/20000] Loss:0.322 Acc: 94.355
Epoch[1700/20000] Loss:0.31 Acc: 94.355
Epoch[1800/20000] Loss:0.3 Acc: 94.355
Epoch[1900/20000] Loss:0.29 Acc: 94.355
Epoch[2000/20000] Loss:0.282 Acc: 94.355
Epoch[2100/20000] Loss:0.274 Acc: 94.355
Epoch[2200/20000] Loss:0.266 Acc: 94.355
Epoch[2300/20000] Loss:0.259 Acc: 94.355
Epoch[2400/20000] Loss:0.252 Acc:

In [10]:
#궁금한 내용
torch.max(prediction.data,1) #values 와 indices가 출력됨. 원하는 결과 위해서는 _(앞내용),predicted(필요한 내용)으로 처리! / #1=차원

torch.return_types.max(
values=tensor([3.7103, 2.0904, 4.2079, 0.7342, 3.9097, 4.2466, 3.2568, 3.1780, 4.2295,
        4.3813, 2.2345, 3.2139, 3.7977, 2.7289, 3.5194, 2.8019, 2.9252, 3.6527,
        1.5284, 3.5443, 5.3087, 3.1315, 2.7272, 1.8528, 4.7221, 2.4035, 4.6492,
        4.9445, 4.7683, 4.8060, 1.5452, 3.8336, 3.2641, 2.2130, 5.5157, 4.9782,
        3.8592, 2.3231, 2.1094, 3.4066, 4.5526, 3.9933, 3.5826, 3.6945, 4.2736,
        3.2645, 1.6821, 4.1212, 3.6881, 3.8922, 1.5725, 4.6310, 1.8523, 3.5894,
        3.1664, 4.4531, 2.4075, 4.6600, 1.0562, 3.6785, 1.8466, 5.2380, 2.3911,
        2.6928, 4.1068, 3.0701, 2.9748, 3.1946, 4.2743, 4.0469, 1.6309, 4.9424,
        5.3694, 3.4625, 4.4675, 6.5779, 6.0969, 2.8912, 2.8575, 2.0587, 2.1490,
        2.2986, 4.5548, 1.4366, 3.4005, 4.3949, 3.1571, 2.6544, 3.4861, 4.2089,
        2.7751, 1.6108, 4.4027, 4.0729, 2.9676, 1.6350, 1.4714, 2.2270, 4.0007,
        3.4821, 2.6513, 1.4111, 4.1274, 3.8307, 3.8960, 3.6448, 4.6778, 2.4753,
        3

In [11]:
_, predicted = torch.max(prediction.data, 1)

total += y_train_tensor.size(0)
correct += (predicted == y_train_tensor).sum().item()

acc = 100 * correct / total

print(acc)

99.19354838709677


테스트

In [12]:
epochs = 5000
total = 0
correct = 0
model.eval()

with torch.no_grad():
    
    prediction = model(X_test_tensor)
    _, predicted = torch.max(prediction.data, 1)
    total += y_test_tensor.size(0)
    correct += (predicted == y_test_tensor).sum().item()
    
    print(f'Test Acc: {round(100 * correct / total, 3)}')

Test Acc: 100.0
